# Tokenizace

V tomto notebooku si v několika krocích vytvoříme program pro tokenizaci textu. Použijeme k tomu pouze základní metody Pythonu a knihovnu re pro regulární výrazy.

Tokenizace je jedním z běžných problémů zpracování přirozeného jazyka. S tokenizací se setkáme v korpusech, v syntéze řeči, během strojového učení apod. Jedná se o rozdělení textu na menší části, přičemž tyto části, kterým říkáme tokeny, mohou být věty, slova, či jinak definované řetězce. Jelikož se jedná o běžnou součást NLP (zpracování přirozeného jazyka), mohli bychom předpokládat, že tokenizace patří mezi dobře popsané problémy, který není třeba řešit. Jak si ale ukážeme dále, s tokenizací to není tak jednoduché a velmi záleží na tom, pro co data zpracováváme a jak definujeme samotné tokeny.

## 1 Problémy tokenizace

Pokud se zaměříme na tokenizaci slov, musíme se rozhodnout, jak budeme ke slovům přistupovat. Pokud narazíme na slovní spojení "Jižní Amerika", budeme jej tokenizovat jako dvě různá slova "Jižní" a "Amerika", nebo zvolíme variantu "Jižní Amerika", jelikož tato dvě slova spolu tvoří význam? Jak v anglickém textu budeme tokenizovat "don't"? Přikloníme se k variantě "don't", "do" a "n't", "dont", nebo úplně jiné variantě? Pokud narazíme na slovo "překladatel-tlumočník", rozhodneme se ho ponechat se spojovníkem nebo rozdělit do slov "překladatel" a "tlumočník"?  A co slovo "dvou-" ve spojení "dvou‑ až třílůžkový pokoj"? Jak budeme nakládat s daty, čísly, zkratkami, internetovými adresami, interpunkcí apod.?

Seznam by mohl dále pokračovat. Z předchozích příkladů můžeme vidět, že problémů definice slova (tokenu), je spousta, přičemž žádný z uvedených příkladů nemá jedno správné řešení, všechna uvedená by byla vhodná v různých kontextech.

V našem případě nebude důležité vyřešit všechny tyto problémy. Zaměříme se pouze na ty, které se vyskytují v textu, jenž budeme tokenizovat.

## 2 S čím budeme pracovat

### 2.1 re

V tomto notebooku použijeme modul re, který slouží pro práci s regulárními výrazy.

Více informací na [re](https://docs.python.org/3/library/re.html).

Pro připomenutí regulárních výrazů vám může pomoct tento [cheatsheet](https://cheatography.com/davechild/cheat-sheets/regular-expressions/). Než výrazy implementujete do svého kódu, můžete si jejich funkčnost vyzkoušet např. na [regex101](https://regex101.com/) (nezapomeňte si v levém boxu přepnout flavor na Python).

## 3 Instalace

Modul re je součástí tzv. The Python Standard Library (standarní knihovny Pythonu), není tedy nutné nic instalovat.

Více informací na [The Python Standard Library](https://docs.python.org/3/library/).

## 4 Import knihoven a modulů

Než budeme moct začít s psaním programu, musíme importovat všechny knihovny a moduly, které budeme potřebovat. Patří mezi ně:

- re

Spusťte následující buňku, modul re se importuje.

**Poznámka:** Po každém zavření a otevření notebooku je nutné všechen kód (tj. i importování) spustit znovu. Výsledky sice zůstanou zobrazeny, obsah proměnných však v paměti nezůstává.

In [2]:
import re

## 5 Nejjednodušší tokenizátor

Nejjednodušší tokenizátor v Pythonu naprogramujeme na třech řádcích díky metodě `split`. Text rozdělíme na slova pomocí mezer.

Nejprve vyzveme uživatele k zadání textu.

In [10]:
text = input('Zadejte text pro tokenizaci: ')

Zadejte text pro tokenizaci:      V polovině ledna se v médiích objevila zpráva, že předsedkyně Poslanecké sněmovny Markéta Pekarová Adamová shání stážistu. „Stážistce či stážistovi nabízíme aktivní zapojení do práce v oblasti mezinárodních vztahů a diplomatických aktivit předsedkyně, seznámení se s chodem kanceláře předsedkyně a také s prostorami a fungováním Poslanecké sněmovny,“ stojí v inzerátu.


Text poté tokenizujeme pomocí metody `split`. Pokud této metodě nespecifikujeme, podle čeho má text rozdělit, použije k rozdělení slov mezeru. V tomto případě je toto chování, které požadujeme, argumenty metody tedy necháme prázdné.

In [11]:
text = text.split()

Tokenizovaný text vypíšeme.

**Poznámka:** Tímto způsobem lze text vypsat pouze v prostředí Jupyter Notebook. Pokud budete programovat v jiných prostředích, použijte standardní `print(text)`.

In [12]:
text

['V',
 'polovině',
 'ledna',
 'se',
 'v',
 'médiích',
 'objevila',
 'zpráva,',
 'že',
 'předsedkyně',
 'Poslanecké',
 'sněmovny',
 'Markéta',
 'Pekarová',
 'Adamová',
 'shání',
 'stážistu.',
 '„Stážistce',
 'či',
 'stážistovi',
 'nabízíme',
 'aktivní',
 'zapojení',
 'do',
 'práce',
 'v',
 'oblasti',
 'mezinárodních',
 'vztahů',
 'a',
 'diplomatických',
 'aktivit',
 'předsedkyně,',
 'seznámení',
 'se',
 's',
 'chodem',
 'kanceláře',
 'předsedkyně',
 'a',
 'také',
 's',
 'prostorami',
 'a',
 'fungováním',
 'Poslanecké',
 'sněmovny,“',
 'stojí',
 'v',
 'inzerátu.']

Jak si můžeme všimnout, text se tokenizoval, dokonce jsme se díky metodě `split` zbavili počátečních mezer. Hlavní problém je ale s interpunkcí. V našem tokenizátoru určitě nebudeme chtít mít interpunkci jako součást předchozího slova, samotná metoda `split` nám tedy nebude stačit.

V případě tokenizace není ideální ani uložení textu do proměnné přes metodu `input`. Zejména v případě, kdy budeme chtít tokenizovat dlouhé texty, se nám bude hodit jiný přístup, abychom text nemuseli kopírovat a následně jej vkládat.

To stejné platí o přístupu k výslednému tokenizovanému textu. Může se nám hodit nechat jej v proměnné a dále ho zpracovávat, měli bychom ale být schopni výstup uložit do souboru.

Než budeme upravovat náš tokenizátor, naučíme se, jak pomocí Pythonu otevřít soubor a přečíst jeho obsah. Ukážeme si, jak obsah souboru přepsat nebo k němu přidat další řádky textu. V neposlední řadě zjistíme, jak vytvořit úplně nový soubor.

### 5.1 Otevření souboru

K otevření souboru v Pythonu slouží metoda `open`. Požaduje dva parametry:

1. název souboru (pokud se soubor nachází ve stejné složce jako program, stačí pouze název s formátem, pokud se soubor nachází jinde, je nutné specifikovat celou cestu),
2. mód, v jakém se soubor otevře (v našem případě `r`= read = ke čtení).

Následující buňku pouze spusťte.

In [8]:
txt = open('uryvek.txt', 'r')

In [9]:
txt

<_io.TextIOWrapper name='uryvek.txt' mode='r' encoding='UTF-8'>

In [10]:
txt2 = txt.read()

In [11]:
txt2

'      V polovině ledna se v médiích objevila zpráva, že předsedkyně Poslanecké sněmovny Markéta Pekarová Adamová shání stážistu. „Stážistce či stážistovi nabízíme aktivní zapojení do práce v oblasti mezinárodních vztahů a diplomatických aktivit předsedkyně, seznámení se s chodem kanceláře předsedkyně a také s prostorami a fungováním Poslanecké sněmovny,“ stojí v inzerátu.\n'

### 5.2 Zavření souboru


In [ ]:
txt.close()

### 5.3 Zápis do již existujícího souboru

### 5.4 Vytvoření nového souboru

V další sekci notebooku proto tokenizátor upravíme pomocí regulárních výrazů. Text pro tokenizaci budeme načítat ze souboru a ukážeme si různé způsoby práce s výsledným tokenizovaným textem.

## 6 Tokenizátor s použitím regulárních výrazů

Naprogramujte tokenizátor, který 

Kod z Programming for Linguists

In [ ]:
"""This program works as a simple segmentation and tokenization tool. It reads a text file
and then outputs the text, one sentence per line, all tokens separated by space. Tokens are:
words, abbreviations ('e.g.', 'Mr.', 'Mrs.', and 'U.S.A.' types), abbreviated forms of verbs (isn't),
punctuation, quotes, brackets, numbers ('55', '55.5', '55,5', '55,000.55', and '55,000.55' types),
numbers together with currencies or percentage signs."""


import re

def prepare_raw(data):
    """Read file, remove file's newline characters and replace them by space. Then split
    the text using space."""

    data = open(data, "r")
    raw = data.read()
    raw = raw.replace("\n", " ")
    raw = raw.split(" ")
    raw = raw[:-1] # used because of standard behaviour of .split() method 
                   # which appends an empty string at the end of the list
    data.close()
    return raw


def newline_char(raw):
    """Append newline characters at the end of the sentences."""

    abbr = re.compile(r'\b(?:[A-Za-z]\.)+|\b(?:[A-Z][a-z]{1,2}\.)+|(?:[A-Z]\.)+') # matches abbreviations such as e.g., Dr., Mrs., or U.S.A.
    end_punct = re.compile(r'(\.|\?|\!)$') # matches .?! punctuation marks (at the end of sentences)
    end_br = re.compile(r'[\.\?\!]+[\)\'"]$') # matches .?! punctuation inside brackets and quotation marks

    for i in range(len(raw)):
       if (re.search(end_punct, raw[i]) or re.search(end_br, raw[i])) and not re.search(abbr, raw[i]):
            raw[i] = raw[i] + "\n"
    return raw


def sentence_list(raw):
    """Makes a list of lists. Each inner list contains a sentence."""

    sentence_per_line = []
    sent = []
    
    for part in raw:
        if part[-1:] != "\n": # append the inner token to the sentence
            sent.append(part)
    
        else:
            sent.append(part[:-1]) # append the last word of the sentence to the sentence
                                   # without the newline character
            sentence_per_line.append(sent) # append the whole sentence to the list of all sentences
            sent = [] # reset the variable for next sentence
    
    return sentence_per_line


def tokenize(sentence_per_line):
    """Tokenize the sentences according to the rules mentioned below."""

    abbr = re.compile(r'\b(?:[A-Za-z]\.)+|\b(?:[A-Z][a-z]{1,2}\.)+|(?:[A-Z]\.)+') # matches abbreviations such as e.g., Dr., Mrs., or U.S.A.
    numbers = re.compile(r'(\d+[.,]?\d+)+') # matches decimal numbers with decimal period, or comma, or both (e.g. 2,000.3)
    end_punct = re.compile(r'(\.|\?|\!)$') # matches .?! punctuation marks (for the end of sentences)
    end_br = re.compile(r'[\.\?\!]+[\)\'"]$') # matches .?! punctuation inside brackets and quotation marks
    brq_front = re.compile(r'^[\"\[\(\{\']') # matches brackets and quotation marks at the beginning of the word
    brq_back = re.compile(r'[\"\]\)\}\']$') # matches brackets and quotations marks at the end of the word

    for lst in sentence_per_line:
        for tok in lst:
            if re.search(end_punct, tok) and len(tok) > 1 and not re.search(abbr, tok):
                token1 = tok[:-1] # separate the word from the punctuation mark
                end = tok[-1] # the punctuation mark itself
                lst.remove(tok) # remove the last word
                lst.append(token1) # append the separated word instead
                lst.append(end) # append the separated punctuation instead

            elif re.search(r'\,$', tok) and len(tok) > 1:
                where = lst.index(tok) # keep an index where to put the comma
                token1 = tok[:-1] # the word itself
                end = tok[-1] # the comma itself
                lst.remove(tok) # remove the last word
                lst.insert(where, token1) # insert the word at the index
                lst.insert(where+1, end) # insert the comma after the word

            elif re.search(end_br, tok) and len(tok) > 1: # search for punctuation inside brackets
                token1 = tok[:-2] # word
                end1 = tok[-2] # punctuation
                end2 = tok[-1] # ending bracket
                lst.remove(tok) # remove the last word
                lst.append(token1) # append the separated word instead
                lst.append(end1) # append the separated punctuation mark
                lst.append(end2) # append the separated bracket
        
            elif re.search(brq_front, tok) and len(tok) > 1: # front brackets or quotations
                where = lst.index(tok) # where to insert (it can be in the middle of the sentence)
                token2 = tok[1:]
                brq = tok[0]
                lst.remove(tok)
                lst.insert(where, brq) # we first want to insert the front bracket or quotation
                lst.insert(where+1, token2) # then insert the word
        
            elif re.search(brq_back, tok) and len(tok) > 1: # back brackets or quotations
                where = lst.index(tok) # where to insert
                token1 = tok[:-1]
                brq = tok[-1]
                lst.remove(tok)
                lst.insert(where, token1) # here we want the word first
                lst.insert(where+1, brq) # and the bracket or the quote last

    return sentence_per_line


def main():
    try:
        inp = input("Enter a text to tokenize: ")
        
        if inp[-4:] != ".txt": # the file needs to be .txt
            print("NotTxtFileError: Enter a .txt file.")
            return 1
        
        raw = prepare_raw(inp)
        newline_appended = newline_char(raw)
        sentence_per_line = sentence_list(newline_appended)
        tokens = tokenize(sentence_per_line)

        for sentences in tokens: # print tokens divided by space
            for toks in sentences:
                print(toks, end=" ")
            print()

        return 0

    except FileNotFoundError:
        print("FileNotFoundError: Enter an existing file.")

main()